In [ ]:
##Домашнее задание по теме «Многослойная нейронная сеть»

In [ ]:
Задание
Постройте модель на основе полносвязных слоёв для классификации Fashion MNIST из библиотеки torchvision (datasets).
Получите качество на тестовой выборке не ниже 88%

Инструкция по выполнению задания

1. Скачайте тренировочную и тестовою часть датасета Fashion MNIST
2. Постройте модель, выбрав стартовую архитектуру
3. Обучите модель и сверьте качество на тестовой части с заданным порогом
4. Изменяйте архитектуру модели пока качество на тестовой части не будет выше порога. 
Вариации архитектуры можно реализовать через изменение количества слоёв, количества нейронов в слоях и использование регуляризации. 
Можно использовать различные оптимизаторы.

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torchvision.transforms as transforms


In [ ]:
1. Скачайте тренировочную и тестовою часть датасета Fashion MNIST 

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем : {device}")

# Гиперпараметры
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.001

# Загрузка данных FashionMNIST
transform = transforms.ToTensor()

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


Используем : cuda


In [ ]:
2. Постройте модель, выбрав стартовую архитектуру

In [15]:
import torch.nn as nn


model = nn.Sequential(
    nn.Flatten(),         # Превращаем 28x28 картинку в вектор 784
    nn.Linear(784, 128),  # Первый скрытый слой (128 нейронов)
    nn.ReLU(),           # Активация ReLU
    nn.Linear(128, 10)    # Выходной слой 
)

print(model)


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=10, bias=True)
)


In [ ]:
 # Функция потерь и оптимизатор

In [16]:
import torch.optim as optim
loss = nn.CrossEntropyLoss()
trainer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [17]:
def train_model():
    for ep in range(EPOCHS):
        train_iters, train_passed = 0, 0
        train_loss, train_acc = 0., 0.
        start = time.time()
        
        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X.view(-1, 28*28))  # Преобразуем картинку в вектор 784 элементов
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        with torch.no_grad():
            for X, y in test:
                y_pred = model(X.view(-1, 28*28))
                l = loss(y_pred, y)
                
                test_loss += l.item()
                test_acc += (y_pred.argmax(dim=1) == y).sum().item()
                test_iters += 1
                test_passed += len(X)
        
        print(f"ep: {ep}, time: {time.time() - start:.3f}, "
              f"train_loss: {train_loss / train_iters:.4f}, train_acc: {train_acc / train_passed:.4f}, "
              f"test_loss: {test_loss / test_iters:.4f}, test_acc: {test_acc / test_passed:.4f}")


In [18]:
train_model()


ep: 0, time: 1.814, train_loss: 0.5579, train_acc: 0.8073, test_loss: 0.4851, test_acc: 0.8251
ep: 1, time: 2.095, train_loss: 0.4066, train_acc: 0.8557, test_loss: 0.4234, test_acc: 0.8499
ep: 2, time: 2.073, train_loss: 0.3681, train_acc: 0.8691, test_loss: 0.3963, test_acc: 0.8591
ep: 3, time: 5.650, train_loss: 0.3415, train_acc: 0.8770, test_loss: 0.3685, test_acc: 0.8678
ep: 4, time: 3.239, train_loss: 0.3204, train_acc: 0.8845, test_loss: 0.3601, test_acc: 0.8717
ep: 5, time: 2.264, train_loss: 0.3037, train_acc: 0.8893, test_loss: 0.3381, test_acc: 0.8810
ep: 6, time: 2.137, train_loss: 0.2923, train_acc: 0.8932, test_loss: 0.3454, test_acc: 0.8777
ep: 7, time: 2.097, train_loss: 0.2790, train_acc: 0.8972, test_loss: 0.3511, test_acc: 0.8737
ep: 8, time: 5.702, train_loss: 0.2712, train_acc: 0.8999, test_loss: 0.3313, test_acc: 0.8816
ep: 9, time: 6.014, train_loss: 0.2622, train_acc: 0.9027, test_loss: 0.3268, test_acc: 0.8851


In [ ]:
#test_acc: 0.8851 на  
#BATCH_SIZE = 64
#EPOCHS = 10
#LEARNING_RATE = 0.001

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем : {device}")

# Гиперпараметры
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.001

# Загрузка данных FashionMNIST
transform = transforms.ToTensor()

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


Используем : cuda


In [ ]:
BatchNorm1d — ускоряет обучение
Dropout — предотвращает переобучение
Больше нейронов в скрытых слоях

In [57]:
import torch.nn as nn

# Улучшенная модель для FashionMNIST
model = nn.Sequential(
    nn.Flatten(),

    nn.Linear(784, 2048),
    nn.BatchNorm1d(2048),
    nn.LeakyReLU(0.1),  # Улучшенная активация
    nn.Dropout(0.4),  # Увеличенный dropout

    nn.Linear(2048, 1024),
    nn.BatchNorm1d(1024),
    nn.LeakyReLU(0.1),
    nn.Dropout(0.4),

    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.LeakyReLU(0.1),
    nn.Dropout(0.3),

    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.LeakyReLU(0.1),
    nn.Dropout(0.3),

    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.SELU(),  # SELU для стабилизации обучения
    nn.Dropout(0.2),  # Чуть меньше dropout

    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.SELU(),
    nn.Dropout(0.2),

    nn.Linear(64, 10)  # Выходной слой 
)

print(model)


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=2048, bias=True)
  (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): LeakyReLU(negative_slope=0.1)
  (4): Dropout(p=0.4, inplace=False)
  (5): Linear(in_features=2048, out_features=1024, bias=True)
  (6): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.1)
  (8): Dropout(p=0.4, inplace=False)
  (9): Linear(in_features=1024, out_features=512, bias=True)
  (10): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): LeakyReLU(negative_slope=0.1)
  (12): Dropout(p=0.3, inplace=False)
  (13): Linear(in_features=512, out_features=256, bias=True)
  (14): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (15): LeakyReLU(negative_slope=0.1)
  (16): Dropout(p=0.3, inplace=False)
  (17): Linear(in_features=256, out_features=12

In [ ]:
# Добавил switch_to_sgd() — она заменяет Adam на SGD после 10 эпох.
# Вызываем её в train_model() после 10-й эпохи.

In [65]:
import torch.optim as optim

loss = nn.CrossEntropyLoss()

# 1. Начинаем с Adam
trainer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 2. Функция смены оптимизатора после нескольких эпох
def switch_to_sgd():
    global trainer  # Меняем глобальную переменную
    trainer = optim.SGD(model.parameters(), lr=LEARNING_RATE / 10, momentum=0.9, weight_decay=1e-4)
    print("🔄 Переключились на SGD!")


In [66]:
import torch.optim as optim
import time

loss = nn.CrossEntropyLoss()
trainer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Функция переключения на SGD
def switch_to_sgd():
    global trainer
    trainer = optim.SGD(model.parameters(), lr=LEARNING_RATE / 10, momentum=0.9, weight_decay=1e-4)
    print("🔄 Переключились на SGD!")

def train_model():
    for ep in range(EPOCHS):
        train_iters, train_passed = 0, 0
        train_loss, train_acc = 0., 0.
        start = time.time()
        
        model.train()
        for X, y in train:
            trainer.zero_grad()
            y_pred = model(X.view(-1, 28*28))  # Преобразуем картинку в вектор 784 элементов
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        with torch.no_grad():
            for X, y in test:
                y_pred = model(X.view(-1, 28*28))
                l = loss(y_pred, y)
                
                test_loss += l.item()
                test_acc += (y_pred.argmax(dim=1) == y).sum().item()
                test_iters += 1
                test_passed += len(X)
        
        print(f"ep: {ep}, time: {time.time() - start:.3f}, "
              f"train_loss: {train_loss / train_iters:.4f}, train_acc: {train_acc / train_passed:.4f}, "
              f"test_loss: {test_loss / test_iters:.4f}, test_acc: {test_acc / test_passed:.4f}")
        
        # Переключаемся на SGD после 10 эпох
        if ep == 10:
            switch_to_sgd()


In [64]:
train_model()

ep: 0, time: 11.143, train_loss: 0.3018, train_acc: 0.8922, test_loss: 0.3267, test_acc: 0.8816
ep: 1, time: 11.067, train_loss: 0.2954, train_acc: 0.8930, test_loss: 0.3137, test_acc: 0.8858
ep: 2, time: 11.761, train_loss: 0.2841, train_acc: 0.8974, test_loss: 0.3168, test_acc: 0.8856
ep: 3, time: 10.877, train_loss: 0.2798, train_acc: 0.8984, test_loss: 0.3105, test_acc: 0.8884
ep: 4, time: 10.799, train_loss: 0.2723, train_acc: 0.9012, test_loss: 0.3039, test_acc: 0.8900
ep: 5, time: 10.766, train_loss: 0.2645, train_acc: 0.9039, test_loss: 0.3251, test_acc: 0.8795
ep: 6, time: 10.854, train_loss: 0.2621, train_acc: 0.9046, test_loss: 0.3075, test_acc: 0.8862
ep: 7, time: 10.792, train_loss: 0.2518, train_acc: 0.9077, test_loss: 0.3044, test_acc: 0.8911
ep: 8, time: 10.769, train_loss: 0.2488, train_acc: 0.9086, test_loss: 0.3008, test_acc: 0.8934
ep: 9, time: 10.798, train_loss: 0.2392, train_acc: 0.9124, test_loss: 0.2893, test_acc: 0.8981


In [ ]:
# loss падает стабильно и на train и на test что говорит что модель обучается, 
# останавливаемсся на 10 эпохах , переобучения нет test_acc: 0.8981